<a href="https://colab.research.google.com/github/tera90223/cow-transport-algorithms/blob/main/MIT_Problem_Set_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 1a: Space Cows
Write your own Greedy Algorithm and Brute Force Algorithm and compare their performance.

In [2]:
# Mount Google Drive to access problem set
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Cows

In [37]:
###########################
# 6.0002 Problem Set 1a: Space Cows
# Name: Chantera Lazard
# Time: 4 hours

import time

#================================
# Part A: Transporting Space Cows
#================================

# Problem 1
def load_cows(filename):
    """
    Read the contents of the given file.  Assumes the file contents contain
    data in the form of comma-separated cow name, weight pairs, and return a
    dictionary containing cow names as keys and corresponding weights as values.

    Parameters:
    filename - the name of the data file as a string

    Returns:
    a dictionary of cow name (string), weight (int) pairs
    """
    # Intialize a dictionary
    cow_dict = {}
    with open(filename) as input_fh:
      # Read each line from file (x,y)
      data = input_fh.readlines()

      for line in data:
        # Split name from weight
        line = line.split(',')
        # Create dictionary with the key being the cow's name &
        # the weight being the value
        cow_dict[line[0]] = int(line[-1].rstrip())
    return cow_dict

## Greedy Algorithm

In [30]:
# Problem 2
def greedy_cow_transport(cows,limit=10):
    """
    Uses a greedy heuristic to determine an allocation of cows that attempts to
    minimize the number of spaceship trips needed to transport all the cows. The
    returned allocation of cows may or may not be optimal.
    The greedy heuristic should follow the following method:

    1. As long as the current trip can fit another cow, add the largest cow that will fit
        to the trip
    2. Once the trip is full, begin a new trip to transport the remaining cows

    Does not mutate the given dictionary of cows.

    Parameters:
    cows - a dictionary of name (string), weight (int) pairs
    limit - weight limit of the spaceship (an int)

    Returns:
    A list of lists, with each inner list containing the names of cows
    transported on a particular trip and the overall list containing all the
    trips
    """
    # Sort the cows dictionary by value (weight) in descending order
    sorted_cows = dict(sorted(cows.items(),
                              key=lambda item: item[1], reverse=True))

    all_trips = []  #list of lists to hold all trips
    used_names = set() #set to keep track of cows already on a trip

    #Iterate through each cow in the sorted list
    for name,weight in sorted_cows.items():
      #validation to make sure not to parse over a cow that is already on a trip
      if name in used_names:
        continue
      else:
        trip = [name] # Start a new trip with this cow
        trip_weight = weight  # Initialize the current trip's weight
        used_names.add(name)  # Mark this cow as on trip

      # Parse through other cows to attempt to add them on current trip
      for name2,weight2 in sorted_cows.items():
        # Only consider unused cows that do not exceed the total weight limit
        if name2 not in used_names and trip_weight + weight2 <=limit:
          trip.append(name2)  # Add cow to current trip
          trip_weight += weight2  # Update total weight
          used_names.add(name2) # Mark this cow as on trip

      all_trips.append(trip)  # Save the completed trip in list of lists

    return all_trips # Return list of lists of all trips

## Brute Force Algorithm

In [41]:
# Problem 3
def brute_force_cow_transport(cows,limit=10):
    """
    Finds the allocation of cows that minimizes the number of spaceship trips
    via brute force.  The brute force algorithm should follow the following method:

    1. Explore all possible ways that the cows can be divided into separate trips
        Use the given get_partitions function in ps1_partition.py to help you!
    2. Select the allocation that minimizes the number of trips without making any trip
        that does not obey the weight limitation

    Does not mutate the given dictionary of cows.

    Parameters:
    cows - a dictionary of name (string), weight (int) pairs
    limit - weight limit of the spaceship (an int)

    Returns:
    A list of lists, with each inner list containing the names of cows
    transported on a particular trip and the overall list containing all the
    trips
    """
    # Initialize variables to track the best (smallest) valid partition
    best_partition = None
    min_trips = len(cows) # Worst case: Each cow goes alone

    # Iterate over every possible partition of the cows
    for partition in get_partitions(cows.keys()):
      valid = True # Flag to track if this partition respects weight limits

      # Check each trip (subset) in the partition
      for trip in partition:
          total_weight = sum(cows[cow] for cow in trip)
          if total_weight > limit: # Trip is overweight; skip this partition
              valid = False
              break

      # If valid and uses fewer trips, store it as the best so far
      if valid and len(partition) < min_trips:
          best_partition = partition
          min_trips = len(partition)

    # Return the partition witht he fewest valid trips
    return best_partition

### Partition Function

In [40]:
# From codereview.stackexchange.com
def partitions(set_):
  """
  Generates all unique partitions of a set using binary combinations.

  Each partitions splits the inplut set into one subset and recursively
  partitions the remaining items.

  Avoids mirrored duplicated by only iterating through half of the 2**n possible
  combinations.

  Parameters: set_: a set of items (i.e., cow names) to partition.

  Yields:
    generator: A generator of partitions, where each partition is a list of
    disjoint sets.
  """
  # Base case: if the set is empty, yield an empty partition
  if not set_:
    yield []
    return

  # Iterate through half of all binary combinations (to avoid duplicates)
  for i in range(2**len(set_)//2):
    parts = [set(), set()] # Initialize two empty subsets

    # Use bitwise logic to assign each item to one of the two subsets
    for item in set_:
        parts[i&1].add(item) # Add item to parts [0] if bit is 0, or parts[1] if bit is 1
        i >>= 1 # Shifts bits right to read the next decision (like floor division //2)

    # Recursively partition the second subset
    for b in partitions(parts[1]):
      # Yield the full partition: current first subset + all partitions of the second
        yield [parts[0]]+b

def get_partitions(set_):
  """
  Converts each set-based partition into a list-of-lists format.

  Parameters:
    set_ (list or set): A collection of items to partition.

  Yields:
    generator: A generator of partitions, where each partition is a list of lists.
  """
  # Loop through each partition generated by the partitions() generator
  for partition in partitions(set_):
      # Convert each set in the partition to a list (for consistency/flexibility)
      yield [list(elt) for elt in partition]

## Greedy Algorithm vs Brute Force Algorithm

In [50]:
# Problem 4
from google.colab import userdata

def compare_cow_transport_algorithms():
    """
    Using the data from ps1_cow_data.txt and the specified weight limit, run your
    greedy_cow_transport and brute_force_cow_transport functions here. Use the
    default weight limits of 10 for both greedy_cow_transport and
    brute_force_cow_transport.

    Print out the number of trips returned by each method, and how long each
    method takes to run in seconds.

    Returns:
    Does not return anything.
    """

    # Load ps1_cow_data.txt
    cows = load_cows(userdata.get('ps1_cow_data.txt'))

    #Greedy Algorithm
    import time # This is present because time is being looked at as a global variable. Can comment out eventually
    start = time.time()
    greedy_algorithm_trips = greedy_cow_transport(cows=cows, limit=10)
    end = time.time()
    num_of_trips = len(greedy_algorithm_trips)
    time = end-start
    print(f"The number of trips returned by the Greedy Algorithm is \
{num_of_trips} and the algorithm took {time} secs to run.")

    #Brute Force Algorithm
    import time
    start = time.time()
    brute_force_algorithm_trips = brute_force_cow_transport(cows=cows, limit=10)
    end = time.time()
    num_of_trips = len(brute_force_algorithm_trips)
    time = end-start
    print(f"The number of trips returned by the Brute Force Algorithm is \
{num_of_trips} and the algorithm took {time} secs to run.")

compare_cow_transport_algorithms()


The number of trips returned by the Greedy Algorithm is 6 and the algorithm took 4.220008850097656e-05 secs to run.
The number of trips returned by the Brute Force Algorithm is 5 and the algorithm took 0.496523380279541 secs to run.


### Summary

#### What were youre results from `compare_cow_transport_algorithms`? Which algorithm runs faster? Why?


> The number of trips returned by the Greedy Algorithm is **6** and the algorithm took **5.340576171875e-05 seconds** to run.

> The number of trips returned by the Brute Force Algorithm is **5** and the algorithm took **0.5293562412261963 seconds** to run.



*   The **Greedy Algorithm** ran significantly faster.
*   The Greedy Algorithm does not sort through all possible combination of cows like the Brute Force Algorithm did. It takes a sorted group of cows by weight, fills the container until it reaches the weight limit, and continues with a new trip.    
*   It runs like an efficient packing line: fast and direct, but not necessarily optimal


#### Does the greedy algorithm return the optimal solution? Why or why not?

*   **No**, It did not return the optimal solution.
*   The greedy algorithm used 6 trips while Brute Force found a better solution in 5 trips.
*   the reason why is because the Greedy Algorithm takes a **heuristic approach**.  It picks the heaviest cow that fits without considering how combinations of smaller cows might be more efficient. It does not look ahead or revise its decisions, which gives it **speed** but also **tunnel vision**.


#### Does the brute force algorithm return the optimal solution? Why or why not?

*   **Yes**, Brute Force Algorithms does return the optimal solution.
*   It explores **all possible combos** of the cows and selects the one with the fewest trips that also satifies the weight limit.
*   However, this comes at the cost of **performance**, especially as the number of cows increases, making it impractical for large datasets.

In summary, the **Greedy Algorithm** is fast although not always optimal. The **Brute Force Algorithm** is optimal but not scalable. Each has trade-offs, and understanding them helps in choosing the right strategy for a given problem.

